# Understanding and testing decompose functions

Premise: Any triclinic vector X in a given 3-D cartesian coordinate system can be decomposed by a cascade of projections into a sum of vectors belonging to the different symmetry classes: $X = X_{tric} + X_{mon} + X_{ort} + X_{tet} + X_{hex} + X_{iso}$.

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
from decompose import *

## Test using the Olivine data

In [2]:
# Elastic stiffness tensor of Olivine used in Browaeys and Chevrot (2004)
Cij = np.array(
    [[192,  66,  60, 0.0, 0.0, 0.0],
    [  66, 160,  56, 0.0, 0.0, 0.0],
    [  60,  56, 272, 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0,  60, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0,  62, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, 0.0,  49]])

In [3]:
# decompose tensor
olivine = decompose_Cij(Cij)

# show
olivine

{'isotropic': array([[194.67,  67.33,  67.33,   0.  ,   0.  ,   0.  ],
        [ 67.33, 194.67,  67.33,   0.  ,   0.  ,   0.  ],
        [ 67.33,  67.33, 194.67,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,  63.67,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,  63.67,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  63.67]]),
 'hexagonal': array([[-21.67,   1.67,  -9.33,   0.  ,   0.  ,   0.  ],
        [  1.67, -21.67,  -9.33,   0.  ,   0.  ,   0.  ],
        [ -9.33,  -9.33,  77.33,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,  -2.67,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,  -2.67,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  , -11.67]]),
 'tetragonal': array([[ 3., -3.,  0.,  0.,  0.,  0.],
        [-3.,  3.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -3.]]),
 'orthorhombic': a

In [4]:
np.around(olivine['hexagonal'], decimals=1)

array([[-21.7,   1.7,  -9.3,   0. ,   0. ,   0. ],
       [  1.7, -21.7,  -9.3,   0. ,   0. ,   0. ],
       [ -9.3,  -9.3,  77.3,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  -2.7,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  -2.7,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. , -11.7]])

In [5]:
Cij

array([[192.,  66.,  60.,   0.,   0.,   0.],
       [ 66., 160.,  56.,   0.,   0.,   0.],
       [ 60.,  56., 272.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  60.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  62.,   0.],
       [  0.,   0.,   0.,   0.,   0.,  49.]])

In [6]:
olivine['orthorhombic'] + olivine['tetragonal'] + np.around(olivine['hexagonal'], decimals=1) + np.around(olivine['isotropic'], decimals=1)

array([[192.,  66.,  60.,   0.,   0.,   0.],
       [ 66., 160.,  56.,   0.,   0.,   0.],
       [ 60.,  56., 272.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  60.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  62.,   0.],
       [  0.,   0.,   0.,   0.,   0.,  49.]])

In [7]:
tensor_to_vector(Cij)

array([192.        , 160.        , 272.        ,  79.19595949,
        84.85281374,  93.33809512, 120.        , 124.        ,
        98.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])

In [8]:
sum_vector = (tensor_to_vector(olivine['orthorhombic']) +
              tensor_to_vector(olivine['tetragonal']) +
              tensor_to_vector(olivine['hexagonal']) +
              tensor_to_vector(olivine['isotropic']))
sum_vector 

array([192.        , 160.        , 272.        ,  79.19595949,
        84.85281374,  93.33809512, 120.        , 124.        ,
        98.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])

### Estimate the percentages of the norm of the different symmetry classes

The different elastic symmetry parts of the tensor can be presented as percentages of the norm of the elastic tensor using the following convention:

$$
percent = 100 \frac{[N^2(X_{tric}) + N^2(X_{mon}) + N^2(X_{ort}) + N^2(X_{tet}) + N^2(X_{hex}) + N^2(X_{iso})]}{N^2(X)}
$$

These norms are defined by the square root of the scalar product (i.e. dot product) of vectors. I have a norm defined by 

$$
N(c) = \sqrt{\langle X, X \rangle} = N(X)
$$

This is the Euclidean norm which is equivalent to:

$$
N(X) = \sqrt{x_1^2 + x_2^2 + ... + x_{21}^2}
$$

These are the different common ways of estimate the Euclidean norm using numpy:


In [9]:
# using linalg.norm
np.linalg.norm(sum_vector)

444.98539301869226

In [10]:
np.sqrt(np.sum(sum_vector**2))

444.98539301869226

In [11]:
# using the dot product inside the sqrt
np.sqrt(np.dot(sum_vector, sum_vector))

444.98539301869226

For estimating the percentages they use the square of the norm which if I understand correctly will be as follows (?)

In [12]:
np.linalg.norm(tensor_to_vector(Cij))**2

198012.00000000003

In [13]:
# Calculate squared norms for each symmetry class using the decomposed vectorr
norm_squared_X = np.linalg.norm(tensor_to_vector(Cij))**2
norm_squared_isotropic = np.linalg.norm(tensor_to_vector(olivine['isotropic']))**2
norm_squared_hexagonal = np.linalg.norm(tensor_to_vector(olivine['hexagonal']))**2
norm_squared_tetragonal = np.linalg.norm(tensor_to_vector(olivine['tetragonal']))**2
norm_squared_orthorhombic = np.linalg.norm(tensor_to_vector(olivine['orthorhombic']))**2
norm_squared_monoclinic = np.linalg.norm(tensor_to_vector(olivine['monoclinic']))**2
norm_squared_remainder = np.linalg.norm(tensor_to_vector(olivine['remainder']))**2
sumaX_squared = norm_squared_isotropic + norm_squared_hexagonal + norm_squared_tetragonal + norm_squared_orthorhombic + norm_squared_monoclinic + norm_squared_remainder

In [14]:
100 * (norm_squared_isotropic / norm_squared_X)

95.71926292345915

In [15]:
100 * (norm_squared_isotropic / sumaX_squared)

95.71622058890803

In [16]:
sumaX_squared, norm_squared_X

(198018.29379999998, 198012.00000000003)

In [17]:
# Calculate squared norms for each symmetry class using the decomposed vector
print("Based on vector squared norms\n",
      f"isotropic:   {100 * norm_squared_isotropic / sumaX_squared:.2f} %\n",
      f"hexagonal:    {100 * norm_squared_hexagonal / sumaX_squared:.2f} %\n",
      f"tetragonal:   {100 * norm_squared_tetragonal / sumaX_squared:.2f} %\n",
      f"orthorhombic: {100 * norm_squared_orthorhombic / sumaX_squared:.2f} %\n",
      f"monoclinic:   {100 * norm_squared_monoclinic / sumaX_squared:.2f} %\n",
      f"others:       {100 * norm_squared_remainder / sumaX_squared:.2f} %")

Based on vector squared norms
 isotropic:   95.72 %
 hexagonal:    3.98 %
 tetragonal:   0.04 %
 orthorhombic: 0.27 %
 monoclinic:   0.00 %
 others:       0.00 %


In [18]:
# Calculate non-squared norms for each symmetry class using the decomposed vector
norm_X = np.linalg.norm(tensor_to_vector(Cij))
norm_isotropic = np.linalg.norm(tensor_to_vector(olivine['isotropic']))
norm_hexagonal = np.linalg.norm(tensor_to_vector(olivine['hexagonal']))
norm_tetragonal = np.linalg.norm(tensor_to_vector(olivine['tetragonal']))
norm_orthorhombic = np.linalg.norm(tensor_to_vector(olivine['orthorhombic']))
norm_monoclinic = np.linalg.norm(tensor_to_vector(olivine['monoclinic']))
norm_remainder = np.linalg.norm(tensor_to_vector(olivine['remainder']))
sumaX = norm_isotropic + norm_hexagonal + norm_tetragonal + norm_orthorhombic + norm_monoclinic + norm_remainder

In [19]:
100 * (norm_isotropic / norm_X)

97.83622178082061

In [20]:
100 * (norm_isotropic / sumaX)

78.3391956141678

In [21]:
print("Based on vector non-squared norms\n",
      f"isotropic:   {100 * norm_isotropic / sumaX:.2f} %\n",
      f"hexagonal:   {100 * norm_hexagonal / sumaX:.2f} %\n",
      f"tetragonal:   {100 * norm_tetragonal / sumaX:.2f} %\n",
      f"orthorhombic: {100 * norm_orthorhombic / sumaX:.2f} %\n",
      f"monoclinic:   {100 * norm_monoclinic / sumaX:.2f} %\n",
      f"others:       {100 * norm_remainder / sumaX:.2f} %")

Based on vector non-squared norms
 isotropic:   78.34 %
 hexagonal:   15.97 %
 tetragonal:   1.53 %
 orthorhombic: 4.17 %
 monoclinic:   0.00 %
 others:       0.00 %


In [22]:
# This is the method currently implemented in the script
calc_percentages(olivine)

{'isotropic': 78.34,
 'hexagonal': 15.97,
 'tetragonal': 1.53,
 'orthorhombic': 4.17,
 'monoclinic': 0.0,
 'remainder': 0.0,
 'anisotropic': 21.66}

In [23]:
# Calculate squared norms for each symmetry class using the decomposed tensors
norm_squared_C = np.linalg.norm(Cij)**2
normC_squared_isotropic = np.linalg.norm(olivine['isotropic'])**2
normC_squared_hexagonal = np.linalg.norm(olivine['hexagonal'])**2
normC_squared_tetragonal = np.linalg.norm(olivine['tetragonal'])**2
normC_squared_orthorhombic = np.linalg.norm(olivine['orthorhombic'])**2
normC_squared_monoclinic = np.linalg.norm(olivine['monoclinic'])**2
normC_squared_remainder = np.linalg.norm(olivine['remainder'])**2
sumaC_squared = normC_squared_isotropic + normC_squared_hexagonal + normC_squared_tetragonal + normC_squared_orthorhombic + normC_squared_monoclinic + normC_squared_remainder

In [24]:
100 * (normC_squared_isotropic / norm_squared_C)

90.84373938282378

In [25]:
100 * (normC_squared_isotropic / sumaC_squared)

95.03361078621785

In [26]:
sumaC_squared, norm_squared_C

(161049.13360000003, 168477.0)

In [27]:
print("Based on tensor squared norms\n",
      f"isotropic:   {100 * normC_squared_isotropic / sumaC_squared:.2f} %\n",
      f"hexagonal:    {100 * normC_squared_hexagonal / sumaC_squared:.2f} %\n",
      f"tetragonal:   {100 * normC_squared_tetragonal / sumaC_squared:.2f} %\n",
      f"orthorhombic: {100 * normC_squared_orthorhombic / sumaC_squared:.2f} %\n",
      f"monoclinic:   {100 * normC_squared_monoclinic / sumaC_squared:.2f} %\n",
      f"others:       {100 * normC_squared_remainder / sumaC_squared:.2f} %")

Based on tensor squared norms
 isotropic:   95.03 %
 hexagonal:    4.61 %
 tetragonal:   0.03 %
 orthorhombic: 0.33 %
 monoclinic:   0.00 %
 others:       0.00 %


In [28]:
# Calculate the non-squared norms for each symmetry class using the decomposed tensors
norm_C = np.linalg.norm(Cij)
normC_isotropic = np.linalg.norm(olivine['isotropic'])
normC_hexagonal = np.linalg.norm(olivine['hexagonal'])
normC_tetragonal = np.linalg.norm(olivine['tetragonal'])
normC_orthorhombic = np.linalg.norm(olivine['orthorhombic'])
normC_monoclinic = np.linalg.norm(olivine['monoclinic'])
normC_remainder = np.linalg.norm(olivine['remainder'])
sumaC = normC_isotropic + normC_hexagonal + normC_tetragonal + normC_orthorhombic + normC_monoclinic + normC_remainder


In [29]:
100 * (normC_isotropic / norm_C)

95.31198213384494

In [30]:
100 * (normC_isotropic / sumaC)

77.14704206471255

In [31]:
print("Based on tensor squared norms\n",
      f"isotropic:   {100 * normC_isotropic / sumaC:.2f} %\n",
      f"hexagonal:   {100 * normC_hexagonal / sumaC:.2f} %\n",
      f"tetragonal:   {100 * normC_tetragonal / sumaC:.2f} %\n",
      f"orthorhombic: {100 * normC_orthorhombic / sumaC:.2f} %\n",
      f"monoclinic:   {100 * normC_monoclinic / sumaC:.2f} %\n",
      f"others:       {100 * normC_remainder / sumaC:.2f} %")

Based on tensor squared norms
 isotropic:   77.15 %
 hexagonal:   16.99 %
 tetragonal:   1.32 %
 orthorhombic: 4.54 %
 monoclinic:   0.00 %
 others:       0.00 %


## Test using the Enstatite data

In [32]:
# Elastic stiffness tensor of Enstatite used in Browaeys and Chevrot (2004)
Cij = np.array(
    [[225,  54,  72, 0.0, 0.0, 0.0],
    [  54, 214,  53, 0.0, 0.0, 0.0],
    [  72,  53, 178, 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0,  78, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0,  82, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, 0.0,  76]])

In [33]:
enstatite = decompose_Cij(Cij)

In [34]:
enstatite

{'isotropic': array([[210.2,  57.4,  57.4,   0. ,   0. ,   0. ],
        [ 57.4, 210.2,  57.4,   0. ,   0. ,   0. ],
        [ 57.4,  57.4, 210.2,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,  76.4,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,  76.4,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,  76.4]]),
 'hexagonal': array([[  5.93,  -0.02,   5.1 ,   0.  ,   0.  ,   0.  ],
        [ -0.02,   5.93,   5.1 ,   0.  ,   0.  ,   0.  ],
        [  5.1 ,   5.1 , -32.2 ,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   3.6 ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   3.6 ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   2.97]]),
 'tetragonal': array([[ 3.37, -3.38,  0.  ,  0.  ,  0.  ,  0.  ],
        [-3.38,  3.37,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , -0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,

In [35]:
# Calculate squared norms for each symmetry class using the decomposed vector
norm_squared_X = np.linalg.norm(tensor_to_vector(Cij))**2
norm_squared_isotropic = np.linalg.norm(tensor_to_vector(enstatite['isotropic']))**2
norm_squared_hexagonal = np.linalg.norm(tensor_to_vector(enstatite['hexagonal']))**2
norm_squared_tetragonal = np.linalg.norm(tensor_to_vector(enstatite['tetragonal']))**2
norm_squared_orthorhombic = np.linalg.norm(tensor_to_vector(enstatite['orthorhombic']))**2
norm_squared_monoclinic = np.linalg.norm(tensor_to_vector(enstatite['monoclinic']))**2
norm_squared_remainder = np.linalg.norm(tensor_to_vector(enstatite['remainder']))**2
sum_norms_squared = norm_squared_isotropic + norm_squared_hexagonal + norm_squared_tetragonal + norm_squared_orthorhombic + norm_squared_monoclinic + norm_squared_remainder

print("Based on vector squared norms\n",
      f"isotropic:   {100 * norm_squared_isotropic / sum_norms_squared:.2f} %\n",
      f"hexagonal:    {100 * norm_squared_hexagonal / sum_norms_squared:.2f} %\n",
      f"tetragonal:   {100 * norm_squared_tetragonal / sum_norms_squared:.2f} %\n",
      f"orthorhombic: {100 * norm_squared_orthorhombic / sum_norms_squared:.2f} %\n",
      f"monoclinic:   {100 * norm_squared_monoclinic / sum_norms_squared:.2f} %\n",
      f"others:       {100 * norm_squared_remainder / sum_norms_squared:.2f} %")

Based on vector squared norms
 isotropic:   99.16 %
 hexagonal:    0.60 %
 tetragonal:   0.04 %
 orthorhombic: 0.20 %
 monoclinic:   0.00 %
 others:       0.00 %


In [36]:
# Calculate non-squared norms for each symmetry class using the decomposed vector
norm_X = np.linalg.norm(tensor_to_vector(Cij))
norm_isotropic = np.linalg.norm(tensor_to_vector(enstatite['isotropic']))
norm_hexagonal = np.linalg.norm(tensor_to_vector(enstatite['hexagonal']))
norm_tetragonal = np.linalg.norm(tensor_to_vector(enstatite['tetragonal']))
norm_orthorhombic = np.linalg.norm(tensor_to_vector(enstatite['orthorhombic']))
norm_monoclinic = np.linalg.norm(tensor_to_vector(enstatite['monoclinic']))
norm_remainder = np.linalg.norm(tensor_to_vector(enstatite['remainder']))
sum_norms = norm_isotropic + norm_hexagonal + norm_tetragonal + norm_orthorhombic + norm_monoclinic + norm_remainder

print("Based on vector non-squared norms\n",
      f"isotropic:   {100 * norm_isotropic / sum_norms:.2f} %\n",
      f"hexagonal:   {100 * norm_hexagonal / sum_norms:.2f} %\n",
      f"tetragonal:   {100 * norm_tetragonal / sum_norms:.2f} %\n",
      f"orthorhombic: {100 * norm_orthorhombic / sum_norms:.2f} %\n",
      f"monoclinic:   {100 * norm_monoclinic / sum_norms:.2f} %\n",
      f"others:       {100 * norm_remainder / sum_norms:.2f} %")

Based on vector non-squared norms
 isotropic:   87.47 %
 hexagonal:   6.82 %
 tetragonal:   1.77 %
 orthorhombic: 3.95 %
 monoclinic:   0.00 %
 others:       0.00 %


In [37]:
norm_X, sum_norms

(473.5599222907276, 539.1342213266997)

In [38]:
norm_X / sum_norms

0.8783711060399634

In [39]:
c = norm_X / sum_norms

In [40]:
print("Based on vector non-squared norms\n",
      f"isotropic:   {c * 100 * norm_isotropic / norm_X:.2f} %\n",
      f"hexagonal:   {c * 100 * norm_hexagonal / norm_X:.2f} %\n",
      f"tetragonal:   {c * 100 * norm_tetragonal / norm_X:.2f} %\n",
      f"orthorhombic: {c * 100 * norm_orthorhombic / norm_X:.2f} %\n",
      f"monoclinic:   {c * 100 * norm_monoclinic / norm_X:.2f} %\n",
      f"others:       {c * 100 * norm_remainder / norm_X:.2f} %")

Based on vector non-squared norms
 isotropic:   87.47 %
 hexagonal:   6.82 %
 tetragonal:   1.77 %
 orthorhombic: 3.95 %
 monoclinic:   0.00 %
 others:       0.00 %


The results using the non-squared Euclidean norm are the ones that come closest to the estimated percentages in the Browaeys and Chevrt paper, but they are still a bit off and there must be another way to estimate this properly. A key observation is that the sum of the norms of the decomposed vectors does not add up to 100% (i.e. has exactly the same values as the norm of the original vector), indicating that the decomposed vectors may not be orthogonal.

In [41]:
norm_decomposed_vectors = [norm_isotropic, norm_hexagonal, norm_tetragonal, norm_orthorhombic, norm_monoclinic, norm_remainder]
norm_decomposed_vectors

[471.55508691986347,
 36.74471662702,
 9.538878340769422,
 21.295539439046856,
 0.0,
 1.0048591735576161e-14]

In [42]:
normalized_decomposed_vectors = [v / sum_norms for v in norm_decomposed_vectors]
normalized_decomposed_vectors

[0.8746524859050169,
 0.06815504409384127,
 0.01769295652814652,
 0.03949951347299539,
 0.0,
 1.8638386023518632e-17]

In [43]:
normalized_decomposed_vectors[0] * (normalized_decomposed_vectors[0] / norm_X) * 100

0.16154597023313283

In [44]:
normalized_decomposed_vectors[0] / norm_X

0.001846973201773715

## Quartz

In [45]:
Cij = np.array([[ 93.52,  18.88,  24.43,  11.  ,   0.  ,   0.  ],
                [ 18.88,  93.52,  24.43, -11.  ,   0.  ,   0.  ],
                [ 24.43,  24.43, 129.83,   0.  ,   0.  ,   0.  ],
                [ 11.  , -11.  ,   0.  ,  62.1 ,   0.  ,   0.  ],
                [  0.  ,   0.  ,   0.  ,   0.  ,  62.1 ,  11.  ],
                [  0.  ,   0.  ,   0.  ,   0.  ,  11.  ,  37.32]])

In [46]:
quartz = decompose_Cij(Cij)

In [47]:
quartz

{'isotropic': array([[115.48,  17.65,  17.65,   0.  ,   0.  ,   0.  ],
        [ 17.65, 115.48,  17.65,   0.  ,   0.  ,   0.  ],
        [ 17.65,  17.65, 115.48,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,  48.91,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,  48.91,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  48.91]]),
 'hexagonal': array([[-21.96,   1.23,   6.78,   0.  ,   0.  ,   0.  ],
        [  1.23, -21.96,   6.78,   0.  ,   0.  ,   0.  ],
        [  6.78,   6.78,  14.35,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,  13.19,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,  13.19,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  , -11.59]]),
 'tetragonal': array([[-0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 'orthorhombic': a

## Test orthogonal_projector

In [48]:
orthogonal_projector('tetragonal')[0:9, 0:9]

array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

In [49]:
np.around(orthogonal_projector('hexagonal')[0:9, 0:9], decimals=2)

array([[ 0.38,  0.38,  0.  ,  0.  ,  0.  ,  0.18,  0.  ,  0.  ,  0.25],
       [ 0.38,  0.38,  0.  ,  0.  ,  0.  ,  0.18,  0.  ,  0.  ,  0.25],
       [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.18,  0.18,  0.  ,  0.  ,  0.  ,  0.75,  0.  ,  0.  , -0.35],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.5 ,  0.5 ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.5 ,  0.5 ,  0.  ],
       [ 0.25,  0.25,  0.  ,  0.  ,  0.  , -0.35,  0.  ,  0.  ,  0.5 ]])

In [50]:
np.around(orthogonal_projector('isotropic')[0:9, 0:9], decimals=2)

array([[ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ]])

In [51]:
np.around(orthogonal_projector('orthorhombic')[0:16, 0:16], decimals=2)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [52]:
np.around(orthogonal_projector('monoclinic')[0:16, 0:16], decimals=1)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [53]:
np.around(orthogonal_projector('monoclinic')[15:, 15:], decimals=1)

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

## Step by step checking

In [54]:
# Elastic stiffness tensor of Enstatite used in Browaeys and Chevrot (2004)
Cij = np.array(
    [[225,  54,  72, 0.0, 0.0, 0.0],
    [  54, 214,  53, 0.0, 0.0, 0.0],
    [  72,  53, 178, 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0,  78, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0,  82, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, 0.0,  76]])

In [55]:
decompose_Cij(Cij)

{'isotropic': array([[210.2,  57.4,  57.4,   0. ,   0. ,   0. ],
        [ 57.4, 210.2,  57.4,   0. ,   0. ,   0. ],
        [ 57.4,  57.4, 210.2,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. ,  76.4,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. ,  76.4,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. ,  76.4]]),
 'hexagonal': array([[  5.93,  -0.02,   5.1 ,   0.  ,   0.  ,   0.  ],
        [ -0.02,   5.93,   5.1 ,   0.  ,   0.  ,   0.  ],
        [  5.1 ,   5.1 , -32.2 ,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   3.6 ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   3.6 ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   2.97]]),
 'tetragonal': array([[ 3.37, -3.38,  0.  ,  0.  ,  0.  ,  0.  ],
        [-3.38,  3.37,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , -0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,

### First calculation (tensor to vector)

In [56]:
tensor_to_vector(Cij)

array([225.        , 214.        , 178.        ,  74.95331881,
       101.82337649,  76.36753237, 156.        , 164.        ,
       152.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])

### second calculation (get projector)

In [57]:
np.around(orthogonal_projector('isotropic')[0:9, 0:9], decimals=2)

array([[ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.2 ,  0.2 ,  0.2 ,  0.09,  0.09,  0.09,  0.13,  0.13,  0.13],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.09,  0.09,  0.09,  0.27,  0.27,  0.27, -0.09, -0.09, -0.09],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ],
       [ 0.13,  0.13,  0.13, -0.09, -0.09, -0.09,  0.2 ,  0.2 ,  0.2 ]])

### third (estimate the vector for the symmetry class)

In [58]:
X_vector = tensor_to_vector(Cij)
M = orthogonal_projector('isotropic')
X_symmetry_class = np.dot(M, X_vector)

X_symmetry_class

array([210.2       , 210.2       , 210.2       ,  81.17585848,
        81.17585848,  81.17585848, 152.8       , 152.8       ,
       152.8       ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ])

In [59]:
C_symmetry_class = np.around(vector_to_tensor(X_symmetry_class), decimals=2)

C_symmetry_class

array([[210.2,  57.4,  57.4,   0. ,   0. ,   0. ],
       [ 57.4, 210.2,  57.4,   0. ,   0. ,   0. ],
       [ 57.4,  57.4, 210.2,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  76.4,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  76.4,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  76.4]])

In [60]:
Cij - C_symmetry_class

array([[ 14.8,  -3.4,  14.6,   0. ,   0. ,   0. ],
       [ -3.4,   3.8,  -4.4,   0. ,   0. ,   0. ],
       [ 14.6,  -4.4, -32.2,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   1.6,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   5.6,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  -0.4]])